In [14]:
!pip install datasets transformers scikit-learn nltk sklearn matplotlib seaborn


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
from google.colab import files
uploaded = files.upload()


Saving data_full.json to data_full.json
Saving benchmark_data.json to benchmark_data.json


In [29]:
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import json
import random
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
clinc_data = load_dataset("clinc_oos", "small", split="train")

clinc_df = pd.DataFrame({
    "text": clinc_data["text"],
    "intent": clinc_data["intent"]
})

with open('benchmark_data.json', 'r') as f:
    snips_data = json.load(f)

snips_intents = []
snips_examples = []
for domain in snips_data['domains']:
    if 'intents' in domain and isinstance(domain['intents'], list):
        for intent in domain['intents']:
            if 'examples' in intent:
                for example in intent['examples']:
                    snips_intents.append(intent['description'])
                    snips_examples.append(example)

snips_df = pd.DataFrame({
    "text": snips_examples,
    "intent": snips_intents
})

final_dataset = pd.concat([clinc_df, snips_df], ignore_index=True)

print(final_dataset.head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                                text  intent
0  can you walk me through setting up direct depo...   108.0
1                 i want to switch to direct deposit   108.0
2                       set up direct deposit for me   108.0
3        how do i go about setting up direct deposit   108.0
4  i need to get my paycheck direct deposited to ...   108.0


In [29]:
snips_df.head(5)

,text,intent


In [30]:
clinc_df.head(5)

,text,intent
0,can you walk me through setting up direct depo...,108
1,i want to switch to direct deposit,108
2,set up direct deposit for me,108
3,how do i go about setting up direct deposit,108
4,i need to get my paycheck direct deposited to ...,108


In [26]:
final_dataset.head(5)

,text,intent
0,can you walk me through setting up direct depo...,108.0
1,i want to switch to direct deposit,108.0
2,set up direct deposit for me,108.0
3,how do i go about setting up direct deposit,108.0
4,i need to get my paycheck direct deposited to ...,108.0


In [27]:
final_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7600 entries, 0 to 7599
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    7600 non-null   object 
 1   intent  7600 non-null   float64
dtypes: float64(1), object(1)
memory usage: 118.9+ KB


In [30]:

final_dataset['text'] = final_dataset['text'].str.lower()
final_dataset['text'] = final_dataset['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

X = final_dataset['text']
y = final_dataset['intent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

model = LogisticRegression(max_iter=200)
model.fit(X_train_tfidf, y_train)

y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.88      0.70      0.78        10
         1.0       1.00      0.90      0.95        10
         2.0       1.00      1.00      1.00        12
         3.0       0.56      1.00      0.71         5
         4.0       0.69      0.92      0.79        12
         5.0       0.86      0.86      0.86         7
         6.0       0.85      0.85      0.85        13
         7.0       0.90      1.00      0.95         9
         8.0       0.92      1.00      0.96        11
         9.0       1.00      0.90      0.95        10
        10.0       0.88      0.88      0.88         8
        11.0       0.75      0.90      0.82        10
        12.0       0.89      1.00      0.94         8
        13.0       1.00      0.83      0.91         6
        14.0       1.00      1.00      1.00         8
        15.0       0.89      0.80      0.84        10
        16.0       0.90      0.75      0.82        12
        17.0       1.00    

In [23]:
def classify_intent(query):
    query = query.lower()
    query_tfidf = vectorizer.transform([query])
    predicted_intent = model.predict(query_tfidf)[0]


    intent_map = {
        57.0: "General Query",
        23.0: "Reservation",
        86.0: "Cooking Query",
        91.0: "Visa Card Application",
        77.0: "General Query",
        82.0: "General Greeting",
        106.0: "Music Request",
        59.0: "Travel/Travel Visa Queries",
        120.0: "Electrical Outlet Information",
        103.0: "Credit Card Application Status",
        101.0: "Vacation Request Status",
        132.0: "Visa Information",
        9.0: "Restaurant Reservation",
        61.0: "Language/Translation Queries",
        125.0: "Shopping List",
        144.0: "Name Query",
        54.0: "Food Storage Information",
        88.0: "Settings Change",
        66.0: "Time Zone Information",
        35.0: "Account Balance",
    108.0: "Direct Deposit Setup",
    5.0: "Weather Query",
    10.0: "Device Control",
    }

    return intent_map.get(predicted_intent, "Unknown")

responses = {
    "General Query": [
        "Could you clarify your question so I can assist you better?",
        "Please provide more details about your query."
    ],
    "Reservation": [
        "I can help you with your reservation. Could you please specify the date and time?",
        "What reservation can I assist you with today?"
    ],
    "Cooking Query": [
        "To bake brownies, preheat your oven to 350°F (175°C) and bake for about 20-25 minutes, or until a toothpick comes out clean.",
        "Brownies typically bake at 350°F for 20-25 minutes."
    ],
    "Visa Card Application": [
        "To apply for a Visa card, visit the Visa website or check with your bank for options.",
        "You can apply for a Visa card online or through your bank."
    ],
    "General Greeting": [
        "Hi there! How can I help you today?",
        "Hello! How can I assist you today?"
    ],
    "Music Request": [
        "Sure! Playing 'Hey Jude' for you now.",
        "Now playing 'Hey Jude'. Enjoy!"
    ],
    "Travel/Travel Visa Queries": [
        "To travel internationally, you may need a visa. Check the requirements for your destination country.",
        "Visa requirements vary by country. Make sure to verify with the embassy."
    ],
    "Electrical Outlet Information": [
        "In Australia, they use Type I electrical plugs with a voltage of 230V and a frequency of 50Hz.",
        "Australia uses Type I outlets. The voltage is 230V, 50Hz."
    ],
    "Credit Card Application Status": [
        "Check your credit card application status by logging into your online account or contacting customer service.",
        "You can track your application status online or by calling customer service."
    ],
    "Vacation Request Status": [
        "Your vacation request is being processed. You should receive an update soon.",
        "We'll let you know once your vacation request has been approved."
    ],
    "Visa Information": [
        "For visa details, refer to the embassy or consulate for your travel destination.",
        "Visit your embassy's website for the latest visa requirements."
    ],
    "Restaurant Reservation": [
        "Could you provide the restaurant name and the date/time for your reservation?",
        "Let me know the restaurant and when you'd like to book."
    ],
    "Language/Translation Queries": [
        "In French, 'hello' is 'Bonjour,' and 'goodbye' is 'Au revoir.'",
        "In Brazil, 'hello' is 'Oi' and 'goodbye' is 'Tchau.'"
    ],
    "Shopping List": [
        "I've added bananas to your shopping list. Would you like to add anything else?",
        "Bananas are now on your shopping list. Need anything else?"
    ],
    "Name Query": [
        "I am SmartBot, your assistant. How can I assist you today?",
        "My name is SmartBot. How can I help?"
    ],
    "Food Storage Information": [
        "Avocados should be kept in the fridge once ripe. They stay fresh for about a week.",
        "Ripe avocados can last up to a week in the fridge."
    ],
    "Settings Change": [
        "Your settings have been reset to default.",
        "The settings have been restored to the original configuration."
    ],
    "Time Zone Information": [
        "Los Angeles is in the Pacific Time Zone (PT). Would you like to know about other time zones?",
        "The time zone for Los Angeles is Pacific Time (PT)."
    ],
    "Direct Deposit Setup": [
        "To set up direct deposit, you'll need to provide your bank account details to your employer.",
        "To switch to direct deposit, contact your HR department and provide them with your bank details."
    ],
    "Account Balance": [
        "To check your account balance, you can log into your online banking portal or mobile app."
    ],
    "Weather Query": [
        "You can check the weather by visiting a weather website or using a weather app."
    ],
    "Device Control": [
        "Sure! I can help you with that. Please confirm which device you want to control."
    ],
}



In [31]:
def chatbot_response(query):
    intent = classify_intent(query)

    if intent in responses:
        return random.choice(responses[intent])
    else:
        return "Sorry, I didn't quite catch that. Could you please rephrase your question?"

def chat_with_bot():
    print("Hello! I am SmartBot. How can I assist you today?")

    while True:
        user_query = input("You: ")

        if user_query.lower() in ["exit", "quit", "bye"]:
            print("Goodbye! Have a great day!")
            break

        response = chatbot_response(user_query)
        print(f"Bot: {response}")

chat_with_bot()


Hello! I am SmartBot. How can I assist you today?
You: hi
Bot: Hi there! How can I help you today?
You: play a music
Bot: Sure! Playing 'Hey Jude' for you now.
You: play a music
Bot: Sure! Playing 'Hey Jude' for you now.
You: could you play a song?
Bot: Sure! Playing 'Hey Jude' for you now.
You: can i hear something
Bot: Sorry, I didn't quite catch that. Could you please rephrase your question?
You: how to bake a cake?
Bot: Brownies typically bake at 350°F for 20-25 minutes.
You: exit
Goodbye! Have a great day!
